In [36]:
import numpy as np
import scipy.io

#load dataset
mat_contents = scipy.io.loadmat("/afs/inf.ed.ac.uk/group/teaching/mlprdata/audio/amp_data.mat")
amp_data = mat_contents['amp_data']
#reshape dataset
row_num = amp_data.shape[0]
remainder = row_num % 21;
amp_data = amp_data[0:(row_num-remainder)];
amp_data = amp_data.reshape((-1,21))

In [ ]:
order = 4

def ls(x,y):
    y=np.dot(x.T,y)
    y=np.reshape(y,(-1,1))
    xx=np.dot(x.T,x)
    xx=np.mat(xx)
    x=xx.I
    c=x*y
    print(c)
    return c

In [8]:
ls([[1]],[[1]])

AttributeError: 'list' object has no attribute 'T'